# Validação do Sistema de Pesos

Este notebook valida o sistema de pesos proposto para roteamento:

1. **ISP Usage Weight** (1.0-1.2): Balanceamento interno
2. **Migration Weight** (0.0-0.2): Evitar conflito com migrações
3. **Link Criticality Weight** (0.0-0.4): Proteger recursos críticos

**Range Total:** 1.0 a 1.8


In [131]:
import pickle
from pathlib import Path
from collections import defaultdict

import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


## 1. Carregar Cenário


In [132]:
# Load scenario
from simulador.core.topology import Topology
from simulador.entities.isp import ISP


scenario_path = Path("output/cenario1.pkl")

with open(scenario_path, 'rb') as f:
    cenario = pickle.load(f)

topology: Topology = cenario.topology
lista_de_isps: list[ISP] = topology.lista_de_isps
disaster_node = 9

alfa = 0.2
beta = 0.2
gamma = 0.4


## 2. Calcular ISP Usage Weights

Para cada ISP, calcula a frequência com que cada link aparece em shortest paths internos.


In [133]:
from simulador.entities.isp import ISP


def calculate_isp_usage_weights(isp_list:list[ISP], alfa:float = 0.2):
    """Calculate ISP usage weights based on shortest path frequency."""
    
    link_ocurrances_in_all_isps = defaultdict(int)
    for isp in isp_list:
        
        for edge in isp.edges:
            link_ocurrances_in_all_isps[edge] += 1
    
    weights_per_isp = { isp.isp_id: {} for isp in isp_list }
    for isp in isp_list:
        for edge in isp.edges:
            normalized = alfa*(link_ocurrances_in_all_isps[edge] -1)/ (len(isp_list) -1)
            weights_per_isp[isp.isp_id][edge] = normalized
            reverse_edge = (edge[1], edge[0])
            weights_per_isp[isp.isp_id][reverse_edge] = normalized
    
    return weights_per_isp

isp_usage_weights = calculate_isp_usage_weights(lista_de_isps, alfa)



## 3. Calcular Migration Weights

Baseado em quais links são usados pelos paths de migração de datacenters.


In [134]:
def calculate_migration_weights(lista_de_isps: list[ISP], beta: float = 0.2):
    """Calculate migration weights based on datacenter migration paths."""
    
    link_count = defaultdict(int)
    total_migration_paths = 0
    
    for isp in lista_de_isps:
        if not hasattr(isp, 'datacenter') or isp.datacenter is None:
            continue
        
        datacenter = isp.datacenter
        src = datacenter.source
        dst = datacenter.destination
        
        # Get paths from ISP's internal paths
        if hasattr(isp, 'caminhos_internos_isp') and isp.caminhos_internos_isp:
            if src in isp.caminhos_internos_isp and dst in isp.caminhos_internos_isp[src]:
                paths = isp.caminhos_internos_isp[src][dst]
                
                for path_info in paths:
                    caminho = path_info["caminho"]
                    total_migration_paths += 1
                    
                    for i in range(len(caminho) - 1):
                        link = (caminho[i], caminho[i + 1])
                        link_count[link] += 1
                        
                        reverse = (caminho[i + 1], caminho[i])
                        link_count[reverse] += 1
    
    if not link_count:
        return {}
    
    max_count = max(link_count.values())
    migration_weights = {}
    
    for link, count in link_count.items():
        normalized = count / max_count if max_count > 0 else 0
        weight = normalized * beta
        migration_weights[link] = weight
    
    return migration_weights


migration_weights = calculate_migration_weights(lista_de_isps, beta)


## 4. Calcular Link Criticality Weights

Baseado em quantas ISPs dependem de cada link (bridges).


In [135]:
def calculate_link_criticality(topology, disaster_node, gamma: float = 0.4):
    """Calculate link criticality based on bridges across all ISPs."""
    
    link_criticality = {}
    
    for isp in topology.lista_de_isps:
        # Create ISP subgraph
        isp_graph = topology.topology.subgraph(isp.nodes).copy()
        for edge in isp.edges:
            if (edge[0] in isp.nodes and edge[1] in isp.nodes 
                and not isp_graph.has_edge(edge[0], edge[1])
                and topology.topology.has_edge(edge[0], edge[1])):
                edge_data = topology.topology[edge[0]][edge[1]].copy()
                isp_graph.add_edge(edge[0], edge[1], **edge_data)
        
        # Remove disaster node
        if disaster_node in isp_graph.nodes():
            isp_graph.remove_node(disaster_node)
        
        # Find bridges
        bridges = list(nx.bridges(isp_graph))
        
        for link in bridges:
            # Forward direction
            if link not in link_criticality:
                link_criticality[link] = {
                    'bridge_count': 0,
                    'isp_list': [],
                    'weight_penalty': 0.0,
                }
            link_criticality[link]['bridge_count'] += 1
            link_criticality[link]['isp_list'].append(isp.isp_id)
            
            # Reverse direction
            reverse = (link[1], link[0])
            if reverse not in link_criticality:
                link_criticality[reverse] = {
                    'bridge_count': 0,
                    'isp_list': [],
                    'weight_penalty': 0.0,
                }
            link_criticality[reverse]['bridge_count'] += 1
            link_criticality[reverse]['isp_list'].append(isp.isp_id)
    
    
    link_criticality_return = {}
    for link, data in link_criticality.items():
        normalized = (data['bridge_count'] )
        link_criticality_return[link] = normalized/len(lista_de_isps)*gamma
    
    return link_criticality_return


link_criticality_weights = calculate_link_criticality(topology, disaster_node, gamma)

## 5. Exemplo: Calcular Peso Total de um Link

Vamos calcular o peso total para um link específico em diferentes ISPs para validar a lógica condicional.


In [136]:
def calculate_weights_by_isps(
    lista_de_isps: list[ISP],
    isp_usage_weights: dict,
    migration_weights: dict,
    link_criticality_weights: dict
    ) -> dict[int, dict[tuple[int, int], dict[str, float]]]:
    link_weights_by_isp: dict[int, dict[tuple[int, int], dict[str, float]]] = {}
    for isp in lista_de_isps:
        isp_id = isp.isp_id
        link_weights_by_isp[isp.isp_id] = {}
        for link in isp.edges:
            reverse_link = (link[1], link[0])
            isp_weight = isp_usage_weights.get(isp_id, {}).get(link, 0.0)
            if isp_weight == 1.0:  # Try reverse
                isp_weight = isp_usage_weights.get(isp_id, {}).get(reverse_link, 0.0)
            
            migration_weight = migration_weights.get(link, 0.0)
            if migration_weight == 0.0:  # Try reverse
                migration_weight = migration_weights.get(reverse_link, 0.0)
            
            criticality_weight = link_criticality_weights.get(link, 0.0)
            if criticality_weight == 0.0:  # Try reverse
                criticality_weight = link_criticality_weights.get(reverse_link, 0.0)
            
            link_components = {
                    'isp_usage': isp_weight,
                    'migration': migration_weight,
                    'criticality': criticality_weight,
                    'total': 1 + isp_weight + migration_weight + criticality_weight
                }
            total_multiplier = 1 + isp_weight + migration_weight + criticality_weight
            link_weights_by_isp[isp.isp_id][link] = link_components
            link_weights_by_isp[isp.isp_id][reverse_link] = link_components

    return link_weights_by_isp

weights_by_link_by_isp = calculate_weights_by_isps(lista_de_isps, isp_usage_weights, migration_weights, link_criticality_weights)


In [137]:
# ============================================================================

def create_weighted_graph(
    graph: nx.Graph,
    isp_id: int,
    weights_by_link_by_isp: dict[int, dict[tuple[int, int], dict[str, float]]]
) -> nx.Graph:
    """Create a copy of the graph with modified edge weights.
    
    Each edge weight is multiplied by the total weight from 3 components:
    - ISP Usage Weight (0.0-0.alfa)
    - Migration Weight (0.0-0.beta)  
    - Link Criticality Weight (0.0-0.gamma, conditional on ISP)
    
    Args:
        graph: Original NetworkX graph
        isp_id: ID of the ISP requesting paths
        isp_usage_weights: dict[isp_id]['weights'][link] = weight
        migration_weights: dict[link] = weight
        link_criticality_weights: dict[link]['penalty_per_isp'][isp_id] = weight
        
    Returns:
        Graph with modified edge weights
    """
    weighted_graph = graph.copy()
    
    for u, v, data in weighted_graph.edges(data=True):
        link = (u, v)
        
        total_multiplier = weights_by_link_by_isp.get(isp_id, {}).get(link, {}).get('total', 1.0)
        # Apply to edge weight (distance)
        original_weight = data.get('weight', 1.0)
        data['weight'] = original_weight * total_multiplier
    
    return weighted_graph



In [138]:
# ============================================================================
# SEPARATED VISUALIZATION FUNCTIONS
# ============================================================================

def plot_isp_topology(
    ax,
    isp,
    topology_graph,
    disaster_node,
    isp_id,
    edge_weights,
    link_frequency
):
    """
    Plot ISP topology on the left side (colored by weights, sized by frequency).
    
    Args:
        ax: Matplotlib axis to plot on
        isp: ISP object
        topology_graph: Full network topology
        disaster_node: Node affected by disaster
        isp_id: ISP ID
        edge_weights: Dict of link -> total weight
        link_frequency: Dict of link -> frequency in shortest paths
    """
    # Get ISP nodes
    isp_nodes_set = set(isp.nodes)
    isp_nodes_active = isp_nodes_set - {disaster_node}
    
    # Build ISP graph
    isp_graph = topology_graph.subgraph(isp_nodes_active).copy()
    for edge in isp.edges:
        if (edge[0] in isp_nodes_active and edge[1] in isp_nodes_active 
            and not isp_graph.has_edge(edge[0], edge[1])
            and topology_graph.has_edge(edge[0], edge[1])):
            edge_data = topology_graph[edge[0]][edge[1]].copy()
            isp_graph.add_edge(edge[0], edge[1], **edge_data)
    
    is_connected = nx.is_connected(isp_graph)
    components = list(nx.connected_components(isp_graph)) if not is_connected else []
    
    # Title
    title = f'ISP {isp_id} - Topologia '
    title += '(PARTICIONADA)' if not is_connected else 'com Pesos'
    ax.set_title(title, fontsize=16, fontweight='bold')
    
    # Layout (consistent with full topology)
    pos = nx.spring_layout(topology_graph, seed=7)
    
    # Background nodes
    all_nodes = list(topology_graph.nodes())
    background_nodes = [n for n in all_nodes if n not in isp_nodes_set and n != disaster_node]
    
    # LAYER 1: Background topology (gray)
    if background_nodes:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=background_nodes,
            node_color='lightgray',
            node_size=500,
            edgecolors='gray',
            linewidths=1,
            alpha=0.3,
            ax=ax
        )
    
    all_edges = list(topology_graph.edges())
    nx.draw_networkx_edges(
        topology_graph, pos,
        edgelist=all_edges,
        edge_color='lightgray',
        width=1,
        alpha=0.3,
        ax=ax
    )
    
    # LAYER 2: ISP nodes (highlighted)
    isp_nodes_list = list(isp_nodes_active)
    if isp_nodes_list:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=isp_nodes_list,
            node_color='lightblue',
            node_size=800,
            edgecolors='black',
            linewidths=2,
            ax=ax
        )
    
    # Datacenter destination node
    if isp.datacenter.destination:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=[isp.datacenter.destination],
            node_color='lightgreen',
            node_size=800,
            edgecolors='black',
            linewidths=2,
            ax=ax
        )
    
    # LAYER 3: Disaster node (red X)
    if disaster_node in isp_nodes_set:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=[disaster_node],
            node_color='red',
            node_size=1000,
            node_shape='X',
            edgecolors='darkred',
            linewidths=3,
            ax=ax,
            label='Nó do Desastre'
        )
    
    # Node labels
    nx.draw_networkx_labels(
        topology_graph, pos,
        font_size=9,
        font_weight='bold',
        ax=ax
    )
    
    # LAYER 4: ISP edges (colored by weight, thickness by frequency)
    edges = list(isp_graph.edges())
    weights_list = [edge_weights.get((u, v), 1.0) for u, v in edges]
    
    # Calculate widths based on frequency
    max_freq = max(link_frequency.values()) if link_frequency else 1
    widths = []
    for u, v in edges:
        link = tuple(sorted([u, v]))
        freq = link_frequency.get(link, 0)
        width = 2 + (freq / max_freq) * 6 if max_freq > 0 else 3
        widths.append(width)
    
    # Color mapping
    vmin = 1.0
    vmax = max(weights_list) if weights_list else 1.8
    
    nx.draw_networkx_edges(
        topology_graph, pos,
        edgelist=edges,
        edge_color=weights_list,
        edge_cmap=plt.cm.RdYlGn_r,
        edge_vmin=vmin,
        edge_vmax=vmax,
        width=widths,
        alpha=0.8,
        ax=ax
    )
    
    # Colorbar
    sm = plt.cm.ScalarMappable(
        cmap=plt.cm.RdYlGn_r, 
        norm=plt.Normalize(vmin=vmin, vmax=vmax)
    )
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Peso Total', rotation=270, labelpad=20, fontsize=12)
    
    # Partition warning
    if not is_connected:
        ax.text(0.5, 0.05, f'[!] Rede particionada em {len(components)} componentes', 
                transform=ax.transAxes,
                ha='center', fontsize=11, color='red', fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
    
    ax.axis('off')
    if disaster_node in isp_nodes_set:
        ax.legend(loc='upper left', fontsize=10)
    
    return is_connected, components


def plot_weight_decomposition(
    ax,
    isp_id,
    edge_weights,
    edge_components,
    is_connected,
    components,
    top_n=10
):
    """
    Plot weight decomposition bar chart on the right side.
    
    Args:
        ax: Matplotlib axis to plot on
        isp_id: ISP ID
        edge_weights: Dict of link -> total weight
        edge_components: Dict of link -> {isp_usage, migration, criticality, total}
        is_connected: Whether ISP is connected
        components: List of connected components (if partitioned)
        top_n: Number of top links to show
    """
    if not edge_weights:
        # Fallback: No weights
        ax.set_title(f'ISP {isp_id} - Sem Dados de Pesos', 
                      fontsize=16, fontweight='bold')
        ax.text(0.5, 0.5, 
                'Pesos não foram calculados para esta ISP',
                ha='center', va='center', fontsize=12,
                transform=ax.transAxes)
        ax.axis('off')
        return
    
    # Title
    title = f'ISP {isp_id} - Top {top_n} Links por Peso (Únicos)'
    if not is_connected:
        title += f' ({len(components)} Componentes)'
    ax.set_title(title, fontsize=16, fontweight='bold')
    
    # Normalize edges to avoid duplicates (always store as (min, max))
    unique_edges = {}
    for (u, v), weight in edge_weights.items():
        normalized_edge = (min(u, v), max(u, v))
        if normalized_edge not in unique_edges:
            unique_edges[normalized_edge] = weight
    
    # Get top N links by weight
    sorted_edges = sorted(unique_edges.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    if not sorted_edges:
        ax.text(0.5, 0.5, 'Sem links para mostrar', 
                ha='center', va='center', fontsize=12,
                transform=ax.transAxes)
        ax.axis('off')
        return
    
    link_names = [f"{u}↔{v}" for (u, v), _ in sorted_edges]  # Use ↔ to show undirected
    
    # Extract weight components (try both directions since edge_components may have either)
    def get_edge_component(link, component_name):
        """Get component value, trying both edge directions."""
        if link in edge_components:
            return edge_components[link][component_name]
        # Try reverse direction
        reverse_link = (link[1], link[0])
        if reverse_link in edge_components:
            return edge_components[reverse_link][component_name]
        return 0.0
    
    isp_usage_vals = [get_edge_component(link, 'isp_usage') for link, _ in sorted_edges]
    migration_vals = [get_edge_component(link, 'migration') for link, _ in sorted_edges]
    criticality_vals = [get_edge_component(link, 'criticality') for link, _ in sorted_edges]
    
    x = range(len(link_names))
    bar_height = 0.6
    
    # Stacked bar chart - build cumulative positions
    base_vals = [1.0] * len(x)
    left_isp = base_vals
    left_migration = [base + isp for base, isp in zip(base_vals, isp_usage_vals)]
    left_criticality = [base + isp + mig for base, isp, mig in zip(base_vals, isp_usage_vals, migration_vals)]
    
    # Draw stacked bars
    ax.barh(x, base_vals, bar_height, label='Base (1.0)', color='lightgray', edgecolor='black', linewidth=0.5)
    ax.barh(x, isp_usage_vals, bar_height, left=left_isp, 
             label='ISP Usage', color='skyblue', edgecolor='black', linewidth=0.5)
    ax.barh(x, migration_vals, bar_height, left=left_migration,
             label='Migration', color='lightgreen', edgecolor='black', linewidth=0.5)
    ax.barh(x, criticality_vals, bar_height, left=left_criticality,
             label='Criticality', color='salmon', edgecolor='black', linewidth=0.5)
    
    # Formatting
    ax.set_yticks(x)
    ax.set_yticklabels(link_names, fontsize=10)
    ax.set_xlabel('Peso Total', fontsize=12, fontweight='bold')
    ax.set_ylabel('Link', fontsize=12, fontweight='bold')
    vmax = max(w for w in edge_weights.values())
    ax.set_xlim(0.9, vmax * 1.05)
    ax.legend(loc='lower right', fontsize=10, framealpha=0.9)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Add total weight values
    for i, (link, total_weight) in enumerate(sorted_edges):
        ax.text(total_weight + 0.02, i, f'{total_weight:.2f}', 
                va='center', fontsize=9, fontweight='bold')
    
    # Add note about unique edges
    ax.text(0.98, 0.02, 'Links únicos (↔ = não-direcionais)', 
            transform=ax.transAxes, fontsize=8,
            ha='right', va='bottom', style='italic',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))


# ============================================================================
# MAIN WRAPPER FUNCTION (combines both plots)
# ============================================================================

def visualize_isp_topology_with_weights(
    isp, 
    topology_graph, 
    disaster_node,
    isp_id,
    weights_by_link_by_isp,
    figsize=(18, 10)
):
    """
    Visualize ISP topology with weights (wrapper for two separate plots).
    
    Args:
        isp: ISP object
        topology_graph: Main topology graph (full network)
        disaster_node: Node to remove (disaster)
        isp_id: ID of the ISP
        weights_by_link_by_isp: Computed weights for all ISPs
        figsize: Figure size tuple
    """
    # === PREPARE DATA ===
    isp_nodes_set = set(isp.nodes)
    isp_nodes_active = isp_nodes_set - {disaster_node}
    
    # Build ISP graph
    isp_graph = topology_graph.subgraph(isp_nodes_active).copy()
    for edge in isp.edges:
        if (edge[0] in isp_nodes_active and edge[1] in isp_nodes_active 
            and not isp_graph.has_edge(edge[0], edge[1])
            and topology_graph.has_edge(edge[0], edge[1])):
            edge_data = topology_graph[edge[0]][edge[1]].copy()
            isp_graph.add_edge(edge[0], edge[1], **edge_data)
    
    is_connected = nx.is_connected(isp_graph)
    components = list(nx.connected_components(isp_graph)) if not is_connected else []
    
    if not is_connected:
        print(f"[!] ISP {isp_id} está particionada após o desastre!")
        print(f"   Componentes: {len(components)}")
        for i, comp in enumerate(components, 1):
            print(f"   - Componente {i}: {len(comp)} nós")
    
    # Calculate link frequency (for thickness)
    link_frequency = defaultdict(int)
    if is_connected:
        nodes = sorted(isp_graph.nodes())
        for src in nodes:
            for dst in nodes:
                if src >= dst:
                    continue
                try:
                    path = nx.shortest_path(isp_graph, src, dst, weight='weight')
                    for i in range(len(path) - 1):
                        link = tuple(sorted([path[i], path[i + 1]]))
                        link_frequency[link] += 1
                except nx.NetworkXNoPath:
                    continue
    else:
        for component in nx.connected_components(isp_graph):
            comp_nodes = sorted(component)
            for src in comp_nodes:
                for dst in comp_nodes:
                    if src >= dst:
                        continue
                    try:
                        path = nx.shortest_path(isp_graph, src, dst, weight='weight')
                        for i in range(len(path) - 1):
                            link = tuple(sorted([path[i], path[i + 1]]))
                            link_frequency[link] += 1
                    except nx.NetworkXNoPath:
                        continue
    
    # Calculate weights
    edge_weights = {}
    edge_components = {}
    for u, v in isp_graph.edges():
        for link in [(u, v), (v, u)]:
            if isp_id in weights_by_link_by_isp and weights_by_link_by_isp[isp_id]:
                weights = weights_by_link_by_isp.get(isp_id, {}).get(link, {})
                edge_weights[link] = weights['total']
                edge_components[link] = weights
            else:
                edge_weights[link] = 1.0
                edge_components[link] = {
                    'isp_usage': 1.0,
                    'migration': 0.0,
                    'criticality': 0.0,
                    'total': 1.0
                }
    
    # === CREATE FIGURE WITH TWO PLOTS ===
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # LEFT PLOT: Topology
    is_connected, components = plot_isp_topology(
        ax1,
        isp,
        topology_graph,
        disaster_node,
        isp_id,
        edge_weights,
        link_frequency
    )
    
    # RIGHT PLOT: Weight Decomposition
    plot_weight_decomposition(
        ax2,
        isp_id,
        edge_weights,
        edge_components,
        is_connected,
        components,
        top_n=len(isp.edges)

    )
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"\n{'='*80}")
    print(f"ESTATÍSTICAS - ISP {isp_id}")
    print(f"{'='*80}")
    print(f"Nós da ISP (total): {len(isp_nodes_set)}")
    print(f"Nós após remover desastre: {len(isp_nodes_active)}")
    print(f"Links da ISP: {isp_graph.number_of_edges()}")
    print(f"Conectada: {'Sim' if is_connected else 'Não'}")
    

In [139]:
# ============================================================================
# PATH EXPLORER: Interactive Source-Destination Path Viewer
# ============================================================================

def plot_paths_on_topology(ax, topology_graph, paths_info, source, destination, disaster_node=None, seed=7):
    """
    Plot paths on topology with directional arrows (thickness by frequency).
    
    Args:
        ax: Matplotlib axis
        topology_graph: NetworkX graph
        paths_info: List of path dictionaries with 'caminho', 'distancia', etc.
        source: Source node
        destination: Destination node
        disaster_node: Optional disaster node to highlight
        seed: Layout seed
    """
    if not paths_info:
        ax.text(0.5, 0.5, 'Nenhum caminho encontrado', 
                ha='center', va='center', fontsize=14, transform=ax.transAxes)
        ax.set_title('Sem Caminhos Disponíveis', fontsize=16, fontweight='bold')
        ax.axis('off')
        return
    
    # Count edge frequency in paths (for thickness)
    edge_frequency = defaultdict(int)
    for path_info in paths_info:
        path = path_info['caminho']
        for i in range(len(path) - 1):
            edge = (path[i], path[i+1])
            edge_frequency[edge] += 1
    
    max_freq = max(edge_frequency.values()) if edge_frequency else 1
    
    # Layout
    pos = nx.spring_layout(topology_graph, seed=seed)
    
    ax.set_title(f'Caminhos: {source} → {destination}', fontsize=16, fontweight='bold')
    
    # LAYER 1: Background nodes and edges
    all_nodes = list(topology_graph.nodes())
    nx.draw_networkx_nodes(
        topology_graph, pos,
        nodelist=all_nodes,
        node_color='lightgray',
        node_size=500,
        edgecolors='gray',
        linewidths=1,
        alpha=0.4,
        ax=ax
    )
    
    all_edges = list(topology_graph.edges())
    nx.draw_networkx_edges(
        topology_graph, pos,
        edgelist=all_edges,
        edge_color='lightgray',
        width=1,
        alpha=0.3,
        arrows=False,
        ax=ax
    )
    
    # LAYER 2: Path edges (with directional arrows, colored by rank)
    # Use colormap for path ranking
    cmap = plt.colormaps['viridis']
    colors = [cmap(i / len(paths_info)) for i in range(len(paths_info))]
    
    for path_idx, path_info in enumerate(paths_info):
        path = path_info['caminho']
        color = colors[path_idx]
        
        for i in range(len(path) - 1):
            edge = (path[i], path[i+1])
            freq = edge_frequency[edge]
            
            # Width based on frequency (2-10 range)
            width = 2 + (freq / max_freq) * 8
            
            # Draw directed edge
            nx.draw_networkx_edges(
                topology_graph, pos,
                edgelist=[edge],
                edge_color=[color],
                width=width,
                alpha=0.7,
                arrows=True,
                arrowsize=20,
                arrowstyle='->',
                connectionstyle='arc3,rad=0.1',
                ax=ax
            )
    
    # LAYER 3: Highlight source and destination
    nx.draw_networkx_nodes(
        topology_graph, pos,
        nodelist=[source],
        node_color='green',
        node_size=1000,
        edgecolors='darkgreen',
        linewidths=3,
        ax=ax,
        label='Origem'
    )
    
    nx.draw_networkx_nodes(
        topology_graph, pos,
        nodelist=[destination],
        node_color='blue',
        node_size=1000,
        edgecolors='darkblue',
        linewidths=3,
        ax=ax,
        label='Destino'
    )
    
    # LAYER 4: Disaster node if present
    if disaster_node is not None:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=[disaster_node],
            node_color='red',
            node_size=900,
            node_shape='X',
            edgecolors='darkred',
            linewidths=3,
            alpha=0.8,
            ax=ax,
            label='Desastre'
        )
    
    # Node labels
    nx.draw_networkx_labels(
        topology_graph, pos,
        font_size=9,
        font_weight='bold',
        ax=ax
    )
    
    ax.legend(loc='upper left', fontsize=10)
    ax.axis('off')


def plot_ranked_paths(ax, paths_info, source, destination):
    """
    Display ranked list of paths on the right side.
    
    Args:
        ax: Matplotlib axis
        paths_info: List of path dictionaries
        source: Source node
        destination: Destination node
    """
    if not paths_info:
        ax.text(0.5, 0.5, 'Nenhum caminho encontrado', 
                ha='center', va='center', fontsize=12, transform=ax.transAxes)
        ax.set_title('Caminhos Ranqueados', fontsize=16, fontweight='bold')
        ax.axis('off')
        return
    
    ax.set_title(f'Caminhos Ranqueados ({len(paths_info)} encontrados)', 
                 fontsize=16, fontweight='bold')
    
    # Prepare data
    path_labels = [f"#{i+1}" for i in range(len(paths_info))]
    distances = [p['distancia'] for p in paths_info]
    hops = [len(p['caminho']) - 1 for p in paths_info]
    
    # Colors matching the topology plot
    cmap = plt.colormaps['viridis']
    colors = [cmap(i / len(paths_info)) for i in range(len(paths_info))]
    
    y_pos = range(len(paths_info))
    
    # Create horizontal bar chart
    bars = ax.barh(y_pos, distances, color=colors, edgecolor='black', linewidth=1.5)
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(path_labels, fontsize=10)
    ax.set_xlabel('Distância (km)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Caminho', fontsize=12, fontweight='bold')
    ax.invert_yaxis()  # First path at top
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Add distance and hop labels
    for i, (dist, hop, path_info) in enumerate(zip(distances, hops, paths_info)):
        # Distance label at end of bar
        ax.text(dist + max(distances) * 0.02, i, f'{dist:.1f} km', 
                va='center', fontsize=9, fontweight='bold')
        
        # Hop count and path on the left
        path_str = ' → '.join(map(str, path_info['caminho']))
        ax.text(-max(distances) * 0.02, i, f'[{hop} hops]  {path_str}', 
                va='center', ha='right', fontsize=8, style='italic')
    
    # Adjust limits to fit labels
    ax.set_xlim(-max(distances) * 0.5, max(distances) * 1.15)


In [140]:
# ============================================================================
# UNIFIED GUI: ISP Topology + Path Explorer (Combined)
# ============================================================================

import ipywidgets as widgets
from IPython.display import display, clear_output
from itertools import islice

def create_unified_isp_gui(
    lista_de_isps,
    topology,
    disaster_node,
    weights_by_link_by_isp
):
    """
    Create unified GUI for ISP visualization and path exploration.
    Paths are computed using ISP-specific weighted graphs.
    
    Args:
        lista_de_isps: List of ISP objects
        topology: Topology object with .topology (NetworkX graph)
        disaster_node: Node affected by disaster
        weights_by_link_by_isp: Computed weights for all ISPs
    """
    # Create output widgets
    output_topology = widgets.Output()  # For ISP topology view
    output_paths = widgets.Output()     # For path explorer view
    
    # ========== ISP Selection ==========
    isp_options = [(f"ISP {isp.isp_id}", isp.isp_id) for isp in lista_de_isps]
    isp_dropdown = widgets.Dropdown(
        options=isp_options,
        value=lista_de_isps[0].isp_id,
        description='ISP:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='200px')
    )
    
    # ========== View Mode Selection ==========
    view_mode = widgets.ToggleButtons(
        options=['ISP Topology', 'Path Explorer'],
        value='ISP Topology',
        description='View:',
        style={'description_width': '60px'},
        button_style='info',
        layout=widgets.Layout(width='400px')
    )
    
    # Helper function to get ISP nodes
    def get_isp_nodes(isp_id):
        """Get nodes allocated to a specific ISP."""
        isp = lista_de_isps[isp_id]
        return sorted(isp.nodes)
    
    # ========== Path Explorer Controls ==========
    # Initialize with first ISP's nodes
    initial_isp_nodes = get_isp_nodes(lista_de_isps[0].isp_id)
    
    source_dropdown = widgets.Dropdown(
        options=initial_isp_nodes,
        value=initial_isp_nodes[0] if initial_isp_nodes else None,
        description='Origem:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='200px')
    )
    
    dest_dropdown = widgets.Dropdown(
        options=initial_isp_nodes,
        value=initial_isp_nodes[1] if len(initial_isp_nodes) > 1 else initial_isp_nodes[0],
        description='Destino:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='200px')
    )
    
    k_slider = widgets.IntSlider(
        value=5,
        min=1,
        max=10,
        step=1,
        description='K Paths:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='300px')
    )
    
    # ========== Disaster Mode Toggle ==========
    disaster_mode_checkbox = widgets.Checkbox(
        value=False,
        description='Disaster Mode (Weighted + Remove Disaster Node)',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='450px')
    )
    
    # ========== Weight Parameter Sliders ==========
    alfa_slider = widgets.FloatSlider(
        value=0.2,
        min=0.0,
        max=0.5,
        step=0.05,
        description='α (ISP):',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='300px'),
        readout_format='.2f'
    )
    
    beta_slider = widgets.FloatSlider(
        value=0.2,
        min=0.0,
        max=0.5,
        step=0.05,
        description='β (Migr):',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='300px'),
        readout_format='.2f'
    )
    
    gamma_slider = widgets.FloatSlider(
        value=0.4,
        min=0.0,
        max=0.8,
        step=0.05,
        description='γ (Crit):',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='300px'),
        readout_format='.2f'
    )
    
    recalc_button = widgets.Button(
        description='🔄 Recalcular Pesos',
        button_style='success',
        layout=widgets.Layout(width='180px')
    )
    
    weights_info_label = widgets.HTML(
        value="<p style='font-size: 10px; color: #666;'>"
              "α: ISP Usage | β: Migration | γ: Link Criticality</p>",
        layout=widgets.Layout(margin='0 0 5px 0')
    )
    
    # ========== Info and Status Labels ==========
    info_label = widgets.HTML(
        value="<h3>🌐 ISP Analysis & Path Explorer</h3>"
              "<p><b>ISP Topology:</b> View ISP-specific weights | "
              "<b>Path Explorer:</b> Paths within ISP subnet only<br>"
              "• Normal Mode: Unweighted paths (physical distance)<br>"
              "• Disaster Mode: Weighted paths + Disaster node removed<br>"
              "• <b>Adjust α, β, γ</b> and click 🔄 to recalculate weights dynamically!</p>",
        layout=widgets.Layout(margin='0 0 15px 0')
    )
    
    status_label = widgets.HTML(
        value="",
        layout=widgets.Layout(margin='10px 0')
    )
    
    # Container for path controls (hidden initially)
    path_controls_row1 = widgets.HBox(
        [source_dropdown, dest_dropdown, k_slider],
        layout=widgets.Layout(display='none', margin='5px 0')
    )
    
    path_controls_row2 = widgets.HBox(
        [disaster_mode_checkbox],
        layout=widgets.Layout(display='none', margin='5px 0')
    )
    
    # Container for weight parameter controls
    weight_params_row = widgets.HBox(
        [alfa_slider, beta_slider, gamma_slider, recalc_button],
        layout=widgets.Layout(margin='10px 0')
    )
    
    # Store current weights (will be recalculated)
    current_weights = {
        'isp_usage': isp_usage_weights,
        'migration': migration_weights,
        'link_criticality': link_criticality_weights,
        'combined': weights_by_link_by_isp
    }
    
    # ========== Weight Recalculation Function ==========
    
    def recalculate_weights(button=None):
        """Recalculate all weights with new alpha, beta, gamma values."""
        nonlocal current_weights
        
        alfa = alfa_slider.value
        beta = beta_slider.value
        gamma = gamma_slider.value
        
        status_label.value = f"<p style='color: blue;'>🔄 Recalculando pesos com α={alfa:.2f}, β={beta:.2f}, γ={gamma:.2f}...</p>"
        print(f"🔄 Recalculating weights with α={alfa:.2f}, β={beta:.2f}, γ={gamma:.2f}")
        print("="*80)
        
        try:
            # Recalculate ISP Usage Weights
            isp_usage_new = calculate_isp_usage_weights(lista_de_isps, alfa)
            
            # Recalculate Migration Weights
            migration_new = calculate_migration_weights(lista_de_isps, beta)
            
            # Recalculate Link Criticality
            link_criticality_new = calculate_link_criticality(topology, disaster_node, gamma)
            
            # Combine all weights
            combined_new = calculate_weights_by_isps(
                lista_de_isps,
                isp_usage_new,
                migration_new,
                link_criticality_new
            )
            
            # Update current weights
            current_weights['isp_usage'] = isp_usage_new
            current_weights['migration'] = migration_new
            current_weights['link_criticality'] = link_criticality_new
            current_weights['combined'] = combined_new
            
            status_label.value = f"<p style='color: green;'>✅ Pesos recalculados com sucesso! α={alfa:.2f}, β={beta:.2f}, γ={gamma:.2f}</p>"
            print(f"✅ Weights recalculated successfully!")
            print(f"   Total weight range: 1.0 to {1.0 + alfa + beta + gamma:.2f}")
            
            # Refresh current view
            if view_mode.value == 'ISP Topology':
                update_isp_topology()
            else:
                update_path_explorer()
                
        except Exception as e:
            status_label.value = f"<p style='color: red;'>❌ Erro ao recalcular pesos: {str(e)}</p>"
            print(f"❌ Error recalculating weights: {str(e)}")
            import traceback
            traceback.print_exc()
    
    # ========== Update Functions ==========
    
    def update_isp_topology():
        """Show ISP topology with weights."""
        with output_topology:
            clear_output(wait=True)
            
            isp_id = isp_dropdown.value
            isp = lista_de_isps[isp_id]
            
            print(f"📊 Loading ISP {isp_id} topology with weights...")
            print("="*80)
            
            visualize_isp_topology_with_weights(
                isp,
                topology.topology,
                disaster_node,
                isp_id,
                current_weights['combined']
            )
    
    def update_isp_node_dropdowns():
        """Update source/destination dropdowns based on selected ISP."""
        isp_id = isp_dropdown.value
        isp_nodes = get_isp_nodes(isp_id)
        
        # Store current selections
        current_source = source_dropdown.value
        current_dest = dest_dropdown.value
        
        # Update options
        source_dropdown.options = isp_nodes
        dest_dropdown.options = isp_nodes
        
        # Restore selections if they're still valid, otherwise reset
        if current_source in isp_nodes:
            source_dropdown.value = current_source
        else:
            source_dropdown.value = isp_nodes[0] if isp_nodes else None
        
        if current_dest in isp_nodes:
            dest_dropdown.value = current_dest
        else:
            dest_dropdown.value = isp_nodes[1] if len(isp_nodes) > 1 else isp_nodes[0]
    
    def update_path_explorer():
        """Show path explorer with normal or disaster-aware paths."""
        with output_paths:
            clear_output(wait=True)
            
            isp_id = isp_dropdown.value
            isp = lista_de_isps[isp_id]
            source = source_dropdown.value
            destination = dest_dropdown.value
            k = k_slider.value
            disaster_mode = disaster_mode_checkbox.value
            
            # Validation
            if source == destination:
                status_label.value = "<p style='color: orange;'>⚠️ Origem e destino devem ser diferentes!</p>"
                print("⚠️ Origem e destino devem ser diferentes!")
                return
            
            # Validate source and destination are in ISP
            if source not in isp.nodes or destination not in isp.nodes:
                status_label.value = "<p style='color: orange;'>⚠️ Origem e destino devem estar na ISP selecionada!</p>"
                print(f"⚠️ Source {source} or destination {destination} not in ISP {isp_id}")
                return
            
            mode_str = "Disaster-Aware (Weighted)" if disaster_mode else "Normal (Unweighted)"
            status_label.value = f"<p style='color: blue;'>🔄 Calculando {k} caminhos [{mode_str}] para ISP {isp_id}: {source} → {destination}...</p>"
            
            try:
                from simulador.core.path_manager import PathManager
                
                # Build ISP subgraph
                isp_subgraph = topology.topology.subgraph(isp.nodes).copy()
                for edge in isp.edges:
                    if (edge[0] in isp.nodes and edge[1] in isp.nodes 
                        and not isp_subgraph.has_edge(edge[0], edge[1])
                        and topology.topology.has_edge(edge[0], edge[1])):
                        edge_data = topology.topology[edge[0]][edge[1]].copy()
                        isp_subgraph.add_edge(edge[0], edge[1], **edge_data)
                
                print(f"✓ Using ISP {isp_id} subgraph: {len(isp.nodes)} nodes, {isp_subgraph.number_of_edges()} edges")
                
                if disaster_mode:
                    # === DISASTER MODE: Weighted graph without disaster node ===
                    print(f"📊 Computing DISASTER-AWARE paths for ISP {isp_id}: {source} → {destination}")
                    print(f"Mode: ISP subnet + Weighted routing + Disaster node removed")
                    print("="*80)
                    
                    # Remove disaster node from ISP subgraph
                    isp_subgraph_no_disaster = isp_subgraph.copy()
                    if disaster_node in isp_subgraph_no_disaster.nodes():
                        isp_subgraph_no_disaster.remove_node(disaster_node)
                        print(f"✓ Removed disaster node {disaster_node} from ISP subgraph")
                    
                    # Apply ISP-specific weights to the subgraph
                    weighted_graph = create_weighted_graph(
                        isp_subgraph_no_disaster,
                        isp_id,
                        current_weights['combined']
                    )
                    print(f"✓ Applied ISP {isp_id} weights (Usage + Migration + Criticality)")
                    
                    # Compute k-shortest paths on weighted ISP subgraph without disaster node
                    paths = list(islice(
                        nx.shortest_simple_paths(weighted_graph, source, destination, weight='weight'),
                        k
                    ))
                    
                    # Build paths_info
                    paths_info = []
                    for path in paths:
                        distance = PathManager.calculate_path_distance(isp_subgraph_no_disaster, path)
                        modulation = PathManager.calculate_modulation_factor(distance)
                        weighted_distance = sum(
                            weighted_graph[path[i]][path[i+1]]['weight']
                            for i in range(len(path) - 1)
                        )
                        paths_info.append({
                            'caminho': path,
                            'distancia': distance,
                            'weighted_distance': weighted_distance,
                            'fator_de_modulacao': modulation
                        })
                    
                    # Visualization
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
                    
                    plot_paths_on_topology(ax1, topology.topology, paths_info, source, destination, disaster_node)
                    ax1.set_title(f'ISP {isp_id} - Disaster-Aware Paths (ISP Subnet): {source} → {destination}', 
                                 fontsize=16, fontweight='bold', color='darkred')
                    
                    plot_ranked_paths_with_weights(ax2, paths_info, source, destination, isp_id)
                    
                    plt.tight_layout()
                    plt.show()
                    
                    status_label.value = f"<p style='color: green;'>✅ {len(paths_info)} disaster-aware path(s) found in ISP {isp_id} subnet!</p>"
                    
                    print(f"\n{'='*80}")
                    print(f"SUMMARY - ISP {isp_id} [DISASTER MODE]")
                    print(f"{'='*80}")
                    print(f"Source: {source}  →  Destination: {destination}")
                    print(f"Paths found: {len(paths_info)}")
                    print(f"Routing: ISP {isp_id} subnet + Weighted + Disaster node {disaster_node} removed")
                    
                else:
                    # === NORMAL MODE: Unweighted, ISP subnet only ===
                    print(f"📊 Computing NORMAL paths for ISP {isp_id}: {source} → {destination}")
                    print(f"Mode: ISP subnet + Unweighted routing (physical distance only)")
                    print("="*80)
                    
                    # Compute k-shortest paths on ISP subgraph (unweighted)
                    paths = list(islice(
                        nx.shortest_simple_paths(isp_subgraph, source, destination, weight='weight'),
                        k
                    ))
                    
                    # Build paths_info
                    paths_info = []
                    for path in paths:
                        distance = PathManager.calculate_path_distance(isp_subgraph, path)
                        modulation = PathManager.calculate_modulation_factor(distance)
                        paths_info.append({
                            'caminho': path,
                            'distancia': distance,
                            'weighted_distance': distance,  # Same as real distance in normal mode
                            'fator_de_modulacao': modulation
                        })
                    
                    # Visualization
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
                    
                    plot_paths_on_topology(ax1, topology.topology, paths_info, source, destination, disaster_node)
                    ax1.set_title(f'ISP {isp_id} - Normal Paths (ISP Subnet): {source} → {destination}', 
                                 fontsize=16, fontweight='bold', color='darkblue')
                    
                    plot_ranked_paths(ax2, paths_info, source, destination)
                    
                    plt.tight_layout()
                    plt.show()
                    
                    status_label.value = f"<p style='color: green;'>✅ {len(paths_info)} normal path(s) found in ISP {isp_id} subnet!</p>"
                    
                    print(f"\n{'='*80}")
                    print(f"SUMMARY - ISP {isp_id} [NORMAL MODE]")
                    print(f"{'='*80}")
                    print(f"Source: {source}  →  Destination: {destination}")
                    print(f"Paths found: {len(paths_info)}")
                    print(f"Routing: ISP {isp_id} subnet + Unweighted (physical distance only)")
                
            except nx.NetworkXNoPath:
                status_label.value = f"<p style='color: red;'>❌ Nenhum caminho encontrado entre {source} e {destination}!</p>"
                print(f"❌ No path found between {source} and {destination}!")
            except Exception as e:
                status_label.value = f"<p style='color: red;'>❌ Erro: {str(e)}</p>"
                print(f"❌ Error: {str(e)}")
                import traceback
                traceback.print_exc()
    
    def on_view_mode_change(change):
        """Handle view mode toggle."""
        mode = change['new']
        
        if mode == 'ISP Topology':
            # Show topology view
            output_topology.layout.display = 'block'
            output_paths.layout.display = 'none'
            path_controls_row1.layout.display = 'none'
            path_controls_row2.layout.display = 'none'
            update_isp_topology()
        else:
            # Show path explorer view
            output_topology.layout.display = 'none'
            output_paths.layout.display = 'block'
            path_controls_row1.layout.display = 'flex'
            path_controls_row2.layout.display = 'flex'
            update_path_explorer()
    
    def on_isp_change(change):
        """Handle ISP selection change."""
        # Always update node dropdowns when ISP changes
        update_isp_node_dropdowns()
        
        if view_mode.value == 'ISP Topology':
            update_isp_topology()
        else:
            update_path_explorer()
    
    def on_path_param_change(change):
        """Handle path parameter changes."""
        if view_mode.value == 'Path Explorer':
            update_path_explorer()
    
    # ========== Attach Callbacks ==========
    view_mode.observe(on_view_mode_change, names='value')
    isp_dropdown.observe(on_isp_change, names='value')
    source_dropdown.observe(on_path_param_change, names='value')
    dest_dropdown.observe(on_path_param_change, names='value')
    k_slider.observe(on_path_param_change, names='value')
    disaster_mode_checkbox.observe(on_path_param_change, names='value')
    recalc_button.on_click(recalculate_weights)
    
    # ========== Layout ==========
    controls_row1 = widgets.HBox([isp_dropdown, view_mode])
    
    gui = widgets.VBox([
        info_label,
        controls_row1,
        weights_info_label,
        weight_params_row,
        path_controls_row1,
        path_controls_row2,
        status_label,
        output_topology,
        output_paths
    ])
    
    # ========== Initialize ==========
    update_isp_topology()
    
    return gui


def plot_ranked_paths_with_weights(ax, paths_info, source, destination, isp_id):
    """
    Display ranked paths with ISP-weighted distance information.
    
    Args:
        ax: Matplotlib axis
        paths_info: List of path dictionaries with 'weighted_distance'
        source: Source node
        destination: Destination node
        isp_id: ISP ID
    """
    if not paths_info:
        ax.text(0.5, 0.5, 'Nenhum caminho encontrado', 
                ha='center', va='center', fontsize=12, transform=ax.transAxes)
        ax.set_title('Caminhos Ranqueados', fontsize=16, fontweight='bold')
        ax.axis('off')
        return
    
    ax.set_title(f'ISP {isp_id} - Caminhos Ranqueados ({len(paths_info)} encontrados)', 
                 fontsize=16, fontweight='bold')
    
    # Prepare data
    path_labels = [f"#{i+1}" for i in range(len(paths_info))]
    distances = [p['distancia'] for p in paths_info]
    weighted_distances = [p.get('weighted_distance', p['distancia']) for p in paths_info]
    hops = [len(p['caminho']) - 1 for p in paths_info]
    
    # Colors matching the topology plot
    cmap = plt.colormaps['viridis']
    colors = [cmap(i / len(paths_info)) for i in range(len(paths_info))]
    
    y_pos = range(len(paths_info))
    
    # Create horizontal bar chart (using weighted distance for ranking)
    bars = ax.barh(y_pos, weighted_distances, color=colors, edgecolor='black', linewidth=1.5)
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(path_labels, fontsize=10)
    ax.set_xlabel('Weighted Distance (ISP-specific)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Caminho (Rank)', fontsize=12, fontweight='bold')
    ax.invert_yaxis()  # First path at top
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Add labels
    for i, (dist, w_dist, hop, path_info) in enumerate(zip(distances, weighted_distances, hops, paths_info)):
        # Weighted distance label
        ax.text(w_dist + max(weighted_distances) * 0.02, i, 
                f'W:{w_dist:.1f} (D:{dist:.1f}km)', 
                va='center', fontsize=9, fontweight='bold')
        
        # Path on the left
        path_str = ' → '.join(map(str, path_info['caminho']))
        ax.text(-max(weighted_distances) * 0.02, i, f'[{hop}h]  {path_str}', 
                va='center', ha='right', fontsize=8, style='italic')
    
    # Adjust limits
    ax.set_xlim(-max(weighted_distances) * 0.5, max(weighted_distances) * 1.15)
    
    # Add legend
    ax.text(0.98, 0.02, 'W=Weighted Distance\nD=Real Distance (km)\nh=hops', 
            transform=ax.transAxes, fontsize=9,
            ha='right', va='bottom',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))


# ============================================================================
# CREATE AND DISPLAY UNIFIED GUI
# ============================================================================

print("🚀 Creating Unified ISP Analysis & Path Explorer GUI...\n")
print("Features:")
print("  - ISP Topology: View ISP-specific weights and decomposition")
print("  - Path Explorer: Paths within selected ISP subnet only")
print("    • Source/Destination dropdowns show only ISP's allocated nodes")
print("    • Normal Mode: Unweighted routing (physical distance)")
print("    • Disaster Mode: Weighted routing + Disaster node removed")
print("  - Dynamic Weight Parameters:")
print("    • α (Alpha): ISP Usage weight [0.0-0.5]")
print("    • β (Beta): Migration weight [0.0-0.5]")
print("    • γ (Gamma): Link Criticality weight [0.0-0.8]")
print("    • Click '🔄 Recalcular Pesos' to apply new values!")
print("\n💡 Select different ISPs to see how each has different allocated resources!")
print("💡 Compare Normal vs Disaster mode to see weight impact!")
print("💡 Experiment with α, β, γ values to see how routing changes!\n")

unified_gui = create_unified_isp_gui(
    lista_de_isps,
    topology,
    disaster_node,
    weights_by_link_by_isp
)

display(unified_gui)


🚀 Creating Unified ISP Analysis & Path Explorer GUI...

Features:
  - ISP Topology: View ISP-specific weights and decomposition
  - Path Explorer: Paths within selected ISP subnet only
    • Source/Destination dropdowns show only ISP's allocated nodes
    • Normal Mode: Unweighted routing (physical distance)
    • Disaster Mode: Weighted routing + Disaster node removed
  - Dynamic Weight Parameters:
    • α (Alpha): ISP Usage weight [0.0-0.5]
    • β (Beta): Migration weight [0.0-0.5]
    • γ (Gamma): Link Criticality weight [0.0-0.8]
    • Click '🔄 Recalcular Pesos' to apply new values!

💡 Select different ISPs to see how each has different allocated resources!
💡 Compare Normal vs Disaster mode to see weight impact!
💡 Experiment with α, β, γ values to see how routing changes!

